# First attempt at sliders: slider and plot placement

I think this is a three step process:
1. get the positioning of the sliders and a plot window working, --- DONE
2. set up the interaction between slider values and the files read in. --- DONE
3. plot the corresponding bands

For now, I will not care about how succinct the code is, I just want to get it working. Simplifying and beautifying the code can come later.  

In [1]:
from ipywidgets import FloatSlider, GridspecLayout, interact, interactive, Layout
from bqplot import *
from tables import *

import bqplot as bq 
import bqplot.pyplot as plt
import numpy as np
# import matplotlib.pyplot as plt

## Functions

1. Define `create_slider` to ... create sliders ... in the end I would like each slider to be created given lists of information, e.g., min and max values, descriptions. Then I would write a function that iterates through these lists and calls the create slider function, then add that slider to the Grid. 
2. Define `get_bands` that creates the string to access arrays in the hdf5 file from the slider values, then gets the bands data. 
3. Define `interactive_figure` which takes slider valuea and calls the `get_bands` function, then plots the bands. Currently this function redraws the figure every time the slider is updated, which is slow and inefficient. Trying to figure out how to not do this. 

In [2]:
def create_slider(min_val, max_val, val_step, default_val, description, layout=Layout(width='auto', height='auto')):
    
    return FloatSlider(min=min_val, max=max_val, step=val_step, value=default_val, 
                       description=description, layout=Layout(height='auto', width='400px'))

In [3]:
def get_bands(potential_height, tp, tsoc, deltaU1, deltaU2):
    
    # Nearest-neighbour hopping parameter fixed. 
    t = -1.0
    # For now keep potential_period fixed.
    potential_period = 2.0
    
    params_path_potential = 'VL' + '{0:.0f}'.format(potential_period) + '_VH' + '{0:.3f}'.format(potential_height)
    params_path_hoppings = '_t' + '{0:.2f}'.format(t) + '_tp' + '{0:.2f}'.format(tp) + \
                           '_tsoc' + '{0:.2f}'.format(tsoc)
    params_path_onsite = '_deltaU1' + '{0:.2f}'.format(deltaU1) + '_deltaU2' + '{0:.2f}'.format(deltaU2)
    
    bands_filename = params_path_potential + params_path_hoppings + params_path_onsite
    
    bands = h5file_data.get_node('/VL2_data', 'bands_'+bands_filename)
    
    return bands

In [4]:
def interactive_figure(potential_height, tp, tsoc, deltaU1, deltaU2):
    
    bands = get_bands(potential_height, tp, tsoc, deltaU1, deltaU2)
    
    fig = plt.figure(title=r'test title')
    
    for band_ind in range(len(bands[0])):
        plt.plot(k_points, bands[:, band_ind])
        
    plt.xlabel(r'k path')
    plt.ylabel(r'E (eV)')
    
    plt.show()


## Open HDF5 data file and set init param values

Also get x-axis data with location of high-symmetry points. 

In [5]:
# Open the HDF5 file. Perhaps I shuold figure out a way to close it after user exits program/

# Get the x-axis data.
h5file_data = open_file('all_data.h5', 'r')
k_points = h5file_data.get_node('/VL2_data', 'k_points')
point_indices = h5file_data.get_node('/VL2_data', 'point_indices')
tick_labels = [r'$\Gamma$', 'X', 'M', 'Y', r'$\Gamma$', 'M']

In [6]:
potential_height_init = 0.0
tp_init = -0.05
tsoc_init = 0.0
deltaU1_init = 0.0
deltaU2_init = 0.0


#plt.figure()
#plt.plot(x, f_test(A=A_init, B=B_init, C=C_init, D=D_init, F=F_init))
#plt.show()

## Manually make sliders

For now, I will just copy and paste the create_slider functions to create them all. Tidy this up later.

In [7]:
potential_height_slider = create_slider(min_val=0.0, max_val=0.5, val_step=0.1,
                                        default_val=potential_height_init, description=r'$V_H$')

tp_slider = create_slider(min_val=-0.5, max_val=0.5, val_step=0.05,
                         default_val=tp_init, description=r"$t'$")

tsoc_slider = create_slider(min_val=-0.2, max_val=0.2, val_step=0.1,
                         default_val=tsoc_init, description=r'$t_{soc}$')

deltaU1_slider = create_slider(min_val=-0.5, max_val=0.5, val_step=0.1,
                               default_val=deltaU1_init, description=r'$\Delta U_1$')

deltaU2_slider = create_slider(min_val=-0.5, max_val=0.5, val_step=0.1,
                               default_val=deltaU2_init, description=r'$\Delta U_2$')

## Set up the grid, add sliders and figure

Again, everything being done manually for now. Tidy up with automated `plot_sliders` (or whatever) function later. 

In [8]:
interactive_plot = interactive(interactive_figure, potential_height=potential_height_slider, 
                               tp=tp_slider, tsoc=tsoc_slider, 
                               deltaU1=deltaU1_slider, deltaU2=deltaU2_slider)
output = interactive_plot.children[-1]
output.layout.height = '600px'
interactive_plot

interactive(children=(FloatSlider(value=0.0, description='$V_H$', layout=Layout(height='auto', width='400px'),…

In [12]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
